In [1]:
import json
from datasets import load_dataset, Dataset
import pandas as pd
import pickle
import pyarrow as pa
import pyarrow.parquet as pq
from datasets import load_dataset
import pandas as pd

In [3]:
# Specify the path to the JSON file
file_path = 'datasets/squad-sr-lat.json'

# Open the JSON file and load its contents
with open(file_path, 'r', encoding='utf8') as file:
    data = json.load(file)

# Now, 'data' contains the parsed JSON data as a Python dictionary
print(data.keys())


dict_keys(['version', 'data'])


In [4]:
len(data["data"])

442

In [5]:
i = 1
titles = []
contexts = []
queryes = []
for article in data['data']:
    title = article['title']
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        currect_queryes = []
        for qa_pair in paragraph['qas']:
            question = qa_pair['question']
            
            currect_queryes.append(question)
            i += 1
        titles.append(title)
        contexts.append(context)
        queryes.append(currect_queryes)
                # print(answer)
i

87176

In [6]:
# from collections import defaultdict
  
dataset_for_eval_df = pd.DataFrame({'title' : titles,
                                    'context' : contexts,
                                  #  'query_id' : dataset_for_eval['id'],
                                    'queries' : queryes})

dataset_for_eval_df.head()
dataset_for_eval_df['queries'][0]

['Kome se u 1858. godini u Lurdu, Francuska, navodno pojavila Deva Marija?',
 'Šta je ispred Glavne zgrade Notr Dame?',
 'Bazilika Svetog srca u Notr Dami je pored koje strukture?',
 'Šta je Grota u Notr Dami?',
 'Šta se nalazi na vrhu Glavne zgrade u Notr Damu?']

In [8]:
dataset_for_eval_df.to_csv('datasets/squad_processed.csv', index=False)

table = pa.Table.from_pandas(dataset_for_eval_df)
pq.write_table(table, 'datasets/squad_processed.parquet')


## ms_marco

In [2]:


marco_ds = load_dataset("microsoft/ms_marco", "v1.1")
data_test = marco_ds['test']
ms_marco = data_test.select_columns(['passages', 'query', 'query_id'])
data_test

Dataset({
    features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
    num_rows: 9650
})

In [3]:

pd.set_option('display.max_colwidth', 300)

msmarco_path = "datasets/msmarco.parquet"
translated_marq_df = pd.read_parquet(msmarco_path, engine='pyarrow')

In [4]:
translated_marq_df

,id,query,passage_text
0,0,Da li ljudska kosa zaustavlja veverice?,"[Hranimo naše veverice u dvorištu tokom jeseni i zime i primetili smo da nekoliko njih ima ćelave delove. Jedna ima mesto bez dlake duž leđa i ispod oba krila. Takođe, druga ima nekoliko ćelavih delova na celom grudnom košu. Sve one jedu i čini se da imaju dobar apetit., Životinje ne podnose mir..."
1,1,Koji su benefiti fosilnih goriva?,[Najveća prednost korišćenja fosilnih goriva je ta što se lako mogu skladištiti i transportovati sa jednog mesta na drugo. Velike rezerve uglja se stoga prenose iz rudnika uglja do industrijskih postrojenja koja su udaljena od rudnika. Nafta se takođe prenosi do udaljenih elektrana kako bi se pr...
2,2,šta je apotema,"[Apotema. Apotema pravilnog mnogougla je segment nacrtan od centra mnogougla (tj. centra kruga koji opisuje mnogougao) do jedne strane, tako da je normalan na stranu. Ako pravilni mnogougao ima površinu od A kvadratnih jedinica, obim od P jedinica, apotemu od a jedinica, tada je površina jednaka..."
3,3,prosečna cena za prilagođeni baldahin,"[Čestitamo! Pronašli ste BuyShade.com, vaš izvor za šatore koji se brzo postavljaju, prekrivače za stolove, prilagođene šatore, štampane zastave i još mnogo toga. Sa stotinama konfiguracija proizvoda i stilova u našem portfoliju i timom vrhunskih dizajnera, specijalizovani smo za pravljenje pril..."
4,4,šta je hardver u računaru,"[Hardver se najbolje opisuje kao uređaj, poput tvrdog diska, koji je fizički povezan sa računarom ili nešto što se može fizički dodirnuti. CD-ROM, monitor računarskog ekrana, štampač i grafička kartica su primeri računarskog hardvera. Hardver služi kao sistem za isporuku softverskih rešenja. Har..."
...,...,...,...
3868,3996,definicija ekonomske suverenosti,"[Na primer, sugerisano nam je da je proizvodna baza princip suverenosti jer stvara bogatstvo i omogućava naciji da se održi. Međutim, postojanje proizvodne baze zavisi, pre svega, od uspostavljanja principa Ekonomske Suverenosti., Offline verzija: PDF. Pojam potrošačke suverenosti Definicija: Id..."
3869,3997,Šta znači ime Lyndsey?,"[Lyndsey je varijanta pravopisa za Lindsay. Ime Lindsay je preneto korišćenje škotskog prezimena, onog Sir Walter de Lindesay koji je ime dobio po teritoriji Lindsey u Lincolnshire-u. Lindsay potiče od staroengleskog lindesig što znači ostrvo lindum ili lincoln's marsh od elemenata lindes što zn..."
3870,3998,plata profesora sa doktoratom,"[Obrazovanje: Doktorat Plata: $36,000 - $122,000 Profesor komunikacija Predaje studentima univerziteta o komunikacijama. Obrazovanje: Doktorat Plata: $33,000 - $114,000 Profesor novinarstva Predaje studentima univerziteta o novinarstvu., Ako profesor sa doktoratom nema iskustvo u nastavi, njegov..."
3871,3999,šta je radionika testiranje kose,"[RADIONIKA. Radionika je metoda dijagnoze i lečenja na daljinu koja koristi specijalno dizajnirane instrumente pomoću kojih praktičar može odrediti osnovne uzroke bolesti unutar živog sistema, bilo da je reč o čoveku, životinji, biljci ili samoj zemlji. Obim radionike u teoriji je neograničen; u..."


In [5]:
id_dict = {row['id']: {'passage_text': row['passage_text'], 'query': row['query']} for _, row in translated_marq_df.iterrows()}
# id_dict

In [3]:

# # Calculate the length of each query
# query_lengths = [len(x) for x in data_test['query']]

# # Find the longest, shortest, and average query lengths
# longest_query = max(query_lengths)
# shortest_query = min(query_lengths)
# average_query_length = sum(query_lengths) / len(query_lengths)

# print(f"Longest query length: {longest_query}")
# print(f"Shortest query length: {shortest_query}")
# print(f"Average query length: {average_query_length:.2f}")

# print(len("sta je to ascci"))

In [6]:
titles = []
queries = []
contexts = []
for index, example in enumerate(data_test):
    # Access individual fields like 'query', 'query_id', and 'passages'
    query_id = str(example['query_id'])
    if query_id in id_dict:
        query = id_dict[query_id]['query']
        is_selected = example['passages']['is_selected']
        passage_text = id_dict[query_id]['passage_text']
        if len(passage_text) != len(is_selected):
            print(f"{len(passage_text)}____{len(is_selected)}")
        for i in range(min(len(is_selected), len(passage_text))):
            titles.append("")
            if is_selected[i] == 1:
                queries.append([query])
                contexts.append(passage_text[i])
            else:
                queries.append([])
                contexts.append(passage_text[i])

print()


11____9
11____9
8____9
7____8
7____8
9____10
8____9
7____8
8____9
7____9
9____10
10____9
5____9
9____10
9____10
8____10
7____8
9____10
9____10
6____10
8____9
8____9
8____9
6____7
8____9
8____9
6____7
4____9
4____7
8____7
4____7
8____9
8____9
11____10
7____8
8____9
7____8
9____10
7____9
7____9
9____10
12____8
8____9
8____9
7____9
8____6
8____10
8____10
7____8
7____8
10____6
7____9
8____9
6____8
8____9
7____10
12____10
6____9
5____6
8____9
6____7
7____8
7____8
7____8
9____10
11____9
4____5
5____6
8____9
10____8
8____9
7____10
10____9
8____9
6____7
8____9
8____9
8____9
7____8
5____10
15____8
9____10
8____9
8____9
8____9
7____9
10____9
9____10
9____10
8____10
8____9
8____9
9____10
7____9
7____9
7____8
7____8
8____9
11____8
10____8
6____7
8____9
7____8
6____7
8____9
7____8
6____10
7____9
8____10
7____9
4____8
9____10
3____8
7____8
9____8
6____8
9____10
8____9
7____9
9____10
7____8
6____7
8____9
7____8
6____9
8____9
8____9
9____10
9____10
6____8
8____9
6____9
8____9
8____9
7____9
7____8
4___

In [7]:
  
dataset_for_eval_df = pd.DataFrame({'title' : titles,
                                    'context' : contexts,
                                  #  'query_id' : dataset_for_eval['id'],
                                    'queries' : queries})

dataset_for_eval_df.head()

,title,context,queries
0,,"Hranimo naše veverice u dvorištu tokom jeseni i zime i primetili smo da nekoliko njih ima ćelave delove. Jedna ima mesto bez dlake duž leđa i ispod oba krila. Takođe, druga ima nekoliko ćelavih delova na celom grudnom košu. Sve one jedu i čini se da imaju dobar apetit.",[]
1,,"Životinje ne podnose miris ljudske kose, pa posipanje barijere od odsečene kose oko vaše bašte, ili lagano unošenje u zemlju prilikom sadnje lukovica, očigledno ima neku vrednost. Čitava stvar me nekako natera da se nasmejem. Nikada mi nije palo na pamet da smo mi ti koji smrde.",[]
2,,Rasporedite malo ljudske kose oko vaših bašti sa povrćem i cvećem. To će oterati veverice jer su ljudi predator veverica. Bolje je ako kosa nije oprana tako da će veverice lako uhvatiti ljudski miris.,[Da li ljudska kosa zaustavlja veverice?]
3,,"1 Možete posuti krvavu brašnu oko vaše bašte takođe. 2 Ne hvatajte i ne premeštajte veverice. 3 Ovo je gubitna bitka jer je populacija veverica izuzetno visoka. 4 Takođe, ako je životinja ženka, postoji velika verovatnoća da ćete je odvojiti od beba koje zavise od nje za preživljavanje.",[]
4,,Gubitak dlake kod veverica može biti uzrokovan šugaravicom ili gljivičnom bolešću. Šugaravica je bolest izazvana mikroskopskim grinjama koje se uvlače ispod kože. Grinja šugaravice kod veverica je zabeležena kako kod lisica tako i kod sivih veverica. Nema izveštaja da je ovaj tip šugaravice pren...,[]


In [8]:
len(dataset_for_eval_df)

31374

In [11]:
table = pa.Table.from_pandas(dataset_for_eval_df)

pq.write_table(table, 'datasets/marco_processed.parquet')

## NATURAL

In [14]:
na_path = "datasets/naquestions.parquet"
translated_na_df = pd.read_parquet(msmarco_path, engine='pyarrow')

In [15]:
translated_na_df

,id,query,passage_text
0,-5501481664893105662,ko je predložio da elektroni pokazuju ponašanje talasa i čestica,"[Kroz rad Maks Planka, Alberta Einštajna, Luj de Broglia, Artura Komptona, Nilsa Bora i mnogih drugih, trenutna naučna teorija tvrdi da sve čestice takođe imaju prirodu talasa (i obrnuto). Ovaj fenomen je potvrđen ne samo za elementarne čestice, već i za složene čestice poput atoma pa čak i mole..."
1,8594030243394572667,koliko senatora ima u Senatu SAD-a,"[Senat Sjedinjenih Američkih Država se sastoji od 100 članova, po dva iz svake od 50 saveznih država. U nastavku je lista trenutnih senatora SAD-a koji sede u 115. Kongresu Sjedinjenih Američkih Država.]"
2,-716176363900512091,promena faze iz gasa u čvrsto se naziva,"[Depozicija je termodinamički proces, faza prelaza u kojoj gas prelazi u čvrsto stanje bez prolaska kroz tečnu fazu. Suprotnost depoziciji je sublimacija i zbog toga se ponekad depozicija naziva desublimacija.]"
3,-5593761806871287545,koja je najduža engleska reč u rečniku,"[Najduža reč u bilo kom od glavnih engleskih jezičkih rečnika je pneumonoultramicroscopicsilicovolcanoconiosis, reč koja se odnosi na bolest pluća koja se dobija udisanjem veoma sitnih čestica silicijuma, specifično sa vulkana; medicinski, isto značenje ima kao silikoza. Reč je namerno skovana d..."
4,2074579308847994444,koliki procenat Zemljine površine je voda,"[Voda je raspoređena širom Zemlje. Većina vode u Zemljinoj atmosferi i kori potiče iz slane morske vode svetskog okeana, dok slatka voda čini samo 2.5% ukupnog. Zbog okeana koji pokrivaju otprilike 78% površine Zemlje reflektujući plavu svetlost, Zemlja izgleda plavo iz svemira, i često se naziv..."
...,...,...,...
3784,57573855796458704,Koji motor se nalazi u Holden V8 super automobilu?,"[Svi automobili moraju imati motor smješten naprijed i pogon na zadnjim točkovima. Svi automobili koriste 5.0-litarski, atmosferski V8 motor sa elektronskim ubrizgavanjem goriva, sposoban da proizvede između 460 i 485 kW (620 - 650 KS). Proizvođači su slobodni da biraju između korišćenja motora ..."
3785,4575828710313790645,kada je Međunarodna svemirska stanica lansirana u svemir,"[Međunarodna svemirska stanica viđena 23. maja 2010. godine sa odlazećeg svemirskog broda Atlantis tokom misije STS-132. Statistike stanice: COSPAR ID 1998-067A, SATCAT broj 25544, Pozivni znak Alfa, Posada stanice: Potpuno posada: 6, Trenutno na brodu: 6 (Ekspedicija 53), Lansiranje 20. novembr..."
3786,2303585950274205305,"Kakav je pas bio Laika, svemirski pas?","[Rasa mešanac, verovatno delom haski (ili delom Samojed) i delom terijer.]"
3787,-6008777350997007702,ko je pevao pesmu 'it's a mean old world',"[`` Mean Old World '' je bluz pesma koju je snimio američki bluz gitarista T - Bone Walker 1942. godine. Opisana je (kao i B - strana singla) kao `` prvi važan bluz snimak na električnoj gitari ''. Tokom godina, interpretirali su je i snimili brojni bluz, džez i rok izvođači.]"


In [23]:
titles = []
queries = []
contexts = []
for index, example in translated_na_df.iterrows():
# Access individual fields like 'query', 'query_id', and 'passages'
    query = example['query']
    passage_text = example['passage_text'][0]
    titles.append("")
    queries.append([query])
    contexts.append(passage_text)

print()

In [24]:
dataset_for_eval_df = pd.DataFrame({'title' : titles,
                                    'context' : contexts,
                                  #  'query_id' : dataset_for_eval['id'],
                                    'queries' : queries})

dataset_for_eval_df.head()

,title,context,queries
0,,"Kroz rad Maks Planka, Alberta Einštajna, Luj de Broglia, Artura Komptona, Nilsa Bora i mnogih drugih, trenutna naučna teorija tvrdi da sve čestice takođe imaju prirodu talasa (i obrnuto). Ovaj fenomen je potvrđen ne samo za elementarne čestice, već i za složene čestice poput atoma pa čak i molek...",[ko je predložio da elektroni pokazuju ponašanje talasa i čestica]
1,,"Senat Sjedinjenih Američkih Država se sastoji od 100 članova, po dva iz svake od 50 saveznih država. U nastavku je lista trenutnih senatora SAD-a koji sede u 115. Kongresu Sjedinjenih Američkih Država.",[koliko senatora ima u Senatu SAD-a]
2,,"Depozicija je termodinamički proces, faza prelaza u kojoj gas prelazi u čvrsto stanje bez prolaska kroz tečnu fazu. Suprotnost depoziciji je sublimacija i zbog toga se ponekad depozicija naziva desublimacija.",[promena faze iz gasa u čvrsto se naziva]
3,,"Najduža reč u bilo kom od glavnih engleskih jezičkih rečnika je pneumonoultramicroscopicsilicovolcanoconiosis, reč koja se odnosi na bolest pluća koja se dobija udisanjem veoma sitnih čestica silicijuma, specifično sa vulkana; medicinski, isto značenje ima kao silikoza. Reč je namerno skovana da...",[koja je najduža engleska reč u rečniku]
4,,"Voda je raspoređena širom Zemlje. Većina vode u Zemljinoj atmosferi i kori potiče iz slane morske vode svetskog okeana, dok slatka voda čini samo 2.5% ukupnog. Zbog okeana koji pokrivaju otprilike 78% površine Zemlje reflektujući plavu svetlost, Zemlja izgleda plavo iz svemira, i često se naziva...",[koliki procenat Zemljine površine je voda]


In [25]:
table = pa.Table.from_pandas(dataset_for_eval_df)

pq.write_table(table, 'datasets/naquestions_processed.parquet')